In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import subprocess
import platform
import re

print(torch.cuda.is_available())
if torch.cuda.is_available():
    print("Cuda is Availabe")
else:
    print("Cuda Can't be found")

# Just the device selection options
device = torch.device(
    "cuda:7" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
# device = torch.device("cpu"
# )
print(f"Using {device} device")

True
Cuda is Availabe
Using cuda:7 device


## 

In [2]:
class LeNet5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.act1 = nn.Tanh()
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.act2 = nn.Tanh()
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.act3 = nn.Tanh()

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(1*1*120, 84)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(84, 10)
        
    def forward(self, x):
        # input 1x28x28, output 6x28x28
        x = self.act1(self.conv1(x))
        # input 6x28x28, output 6x14x14
        x = self.pool1(x)
        # input 6x14x14, output 16x10x10
        x = self.act2(self.conv2(x))
        # input 16x10x10, output 16x5x5
        x = self.pool2(x)
        # input 16x5x5, output 120x1x1
        x = self.act3(self.conv3(x))
        # input 120x1x1, output 84
        x = self.act4(self.fc1(self.flat(x)))
        # input 84, output 10
        x = self.fc2(x)
        return x

In [3]:
# Load MNIST data
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0,), (128,)),
])
train = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
test = torchvision.datasets.MNIST('data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=100)
testloader = torch.utils.data.DataLoader(test, shuffle=True, batch_size=100)

import os

optimizer = None
loss_fn = None

def load_model():
    global model, optimizer, loss_fn
    print (os.path.curdir)
    model = LeNet5().to(device)
    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.CrossEntropyLoss()
    
    MODEL_FILENAME='/data/home/snigdhas/SysMLProject/PruningTest-dev/model.sav'
    if (os.path.isfile(MODEL_FILENAME) != True):
        __train__(model, 10)
        torch.save(model.state_dict(),MODEL_FILENAME)
        print (f'Model saved to {MODEL_FILENAME}')
    else:
        model.load_state_dict(torch.load(MODEL_FILENAME,map_location=device))
        model.to(device)
        model.eval()

def __test__ (model__)-> float:
    model__.eval()
    acc = 0
    count = 0
    for X_batch, y_batch in testloader:
        X_batch = X_batch.to(device); y_batch = y_batch.to(device)
        y_pred = model__(X_batch)
        acc += (torch.argmax(y_pred, 1) == y_batch).float().sum()
        count += len(y_batch)
    acc = acc / count
    acc__ = acc.cpu()
    return acc__.numpy()*100
    

def __train__ (model__, n_epochs=10):
    for epoch in range(n_epochs):
        model__.train()
        for X_batch, y_batch in trainloader:
            X_batch = X_batch.to(device); y_batch = y_batch.to(device)
            y_pred = model__(X_batch)
            loss = loss_fn(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print("Epoch %d: model accuracy %.2f%%" % (epoch, __test__(model__)))

In [4]:
#Static Info
def get_chipset_info():
    try:
        output = subprocess.check_output(["lscpu"]).decode("utf-8")
        for line in output.splitlines():
            if "Model name:" in line:
                chipset_name = line.split(":", 1)[1].strip()
                return chipset_name
    except Exception as e:
        return f"Error: {e}"
    

def get_arch():
    # Get system platform information
    return platform.machine()

def get_cache_sizes():
    cache_info = {}

    # Get L1 cache size
    output = subprocess.check_output(["lscpu"]).decode("utf-8")
    for line in output.splitlines():
        if "L1d cache:" in line:
            cache_info["L1_Size"] = line.split(":", 1)[1].strip()

    # Get L2 cache size
    output = subprocess.check_output(["lscpu"]).decode("utf-8")
    for line in output.splitlines():
        if "L2 cache:" in line:
            cache_info["L2_Size"] = line.split(":", 1)[1].strip()

    # Return L1 and L2 cache sizes
    return cache_info.get("L1_Size"), cache_info.get("L2_Size")

def get_ddr_info():
    try:
        output = subprocess.check_output(["sudo", "dmidecode", "--type", "memory"]).decode("utf-8")
        
        ddr_name = None
        ddr_brand = None
        ddr_speed = None
        ddr_info = []

        for line in output.splitlines():
            if "Memory Device" in line:
                ddr_name = None
                ddr_brand = None
                ddr_speed = None
            elif "Manufacturer:" in line:
                ddr_brand = line.split(":", 1)[1].strip()
            elif "Type:" in line:
                ddr_name = line.split(":", 1)[1].strip()
            elif "Speed:" in line:
                ddr_speed = line.split(":", 1)[1].strip()
            if ddr_name and ddr_brand and ddr_speed:
                ddr_info.append({"DDR_Name": ddr_name, "DDR_Brand": ddr_brand, "DDR_Speed": ddr_speed})
                ddr_name = None
                ddr_brand = None
                ddr_speed = None

        if ddr_info:
            return ddr_info
        else:
            return "DDR information not found"
    except Exception as e:
        return f"Error: {e}"
    
#Dynamic Info
def get_cpu_frequencies():
    try:
        with open('/proc/cpuinfo', 'r') as f:
            cpuinfo = f.read()

        cpu_frequencies = {}
        current_core = None

        for line in cpuinfo.splitlines():
            if line.startswith('processor'):
                current_core = line.split(':')[1].strip()
            elif line.startswith('cpu MHz'):
                if current_core:
                    freq = line.split(':')[1].strip()
                    cpu_frequencies[current_core] = freq

        return cpu_frequencies

    except Exception as e:
        return f"Error: {e}"

def get_gpu_load_and_memory_used():
    try:
        output = subprocess.check_output(["gpustat"]).decode("utf-8")
        
        gpu_info = []

        for line in output.splitlines():
            if "|" in line:
                parts = line.strip().split("|")
                gpu_name = parts[0].strip()
                gpu_load = parts[1].strip().split(",")[1].strip().split(" ")[0]
                gpu_memory_used = parts[2].strip().split("/")[0].strip()
                gpu_memory_total = parts[2].strip().split("/")[1].strip().split(" ")[0]
                gpu_info.append({"GPU_Load": gpu_load, "GPU_Memory_Used": gpu_memory_used, "GPU_Total_Memory": gpu_memory_total})
        
        return gpu_info

    except Exception as e:
        return f"Error: {e}"

def read_psi_info(subsystem):
    psi_file = f"/proc/pressure/{subsystem}"
    if not os.path.exists(psi_file):
        print(f"Error: {psi_file} does not exist")
        return None

    with open(psi_file, 'r') as file:
        line = file.read().strip()
        total_match = re.search(r'total=(\d+)', line)
        if total_match:
            total_value = float(total_match.group(1))
        else:
            total_value = None
        
        avg10_match = re.search(r'avg10=(\d+\.\d+)', line)
        avg60_match = re.search(r'avg60=(\d+\.\d+)', line)
        avg300_match = re.search(r'avg300=(\d+\.\d+)', line)

        psi_info = {
            "total": total_value,
            "avg10": float(avg10_match.group(1)) if avg10_match else None,
            "avg60": float(avg60_match.group(1)) if avg60_match else None,
            "avg300": float(avg300_match.group(1)) if avg300_match else None
        }
        
        return psi_info
    
def get_cache_info(model_name):
    try:
        cache_info = {}
        cache_info["L1_Miss_Rate"] = "N/A"
        cache_info["L2_Miss_Rate"] = "N/A"

        if "Intel" in model_name:
            perf_command = ["timeout", "5s", "perf", "stat","-I", "1000", "-e", "L1-dcache-load-misses,L1-dcache-loads,L1-dcache-stores,L1-dcache-store-misses,l2_rqsts.all_demand_miss,l2_rqsts.all_demand_references"]
        elif "AMD" in model_name:
            perf_command = ["timeout", "5s", "perf", "stat","-I", "1000", "-e", "L1-dcache-load-misses,L1-dcache-loads,L1-dcache-stores,L1-dcache-store-misses,\
                            l2_request_g1.all_no_prefetch,l2_pf_hit_l2,l2_pf_miss_l2_hit_l3,l2_pf_miss_l2_l3\
                            l2_cache_req_stat.ic_dc_miss_in_l2 ,l2_pf_miss_l2_hit_l3,l2_pf_miss_l2_l3"]
        else:
            perf_command=[]

        pipe = subprocess.Popen(perf_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        pipe.terminate()
        output, stderr = pipe.communicate()
        if "Intel" in model_name:
            for line in output.splitlines():
                if "L1-dcache-load-misses" in line:
                    L1_dcache_load_misses = int(line.split(",")[1].strip())
                elif "L1-dcache-loads" in line:
                    L1_dcache_loads = int(line.split(",")[1].strip())
                elif "L1-dcache-store-misses" in line:
                    L1_dcache_store_misses = int(line.split(",")[1].strip())
                elif "L1-dcache-store" in line:
                    L1_dcache_stores = int(line.split(",")[1].strip())
                elif "l2_rqsts.all_demand_miss" in line:
                    L2_all_demand_misses = int(line.split(",")[1].strip())
                elif "l2_rqsts.all_demand_references" in line:
                    L2_all_demand_references = int(line.split(",")[1].strip())
            
        elif "AMD" in model_name:
            for line in output.splitlines():
                if "L1-dcache-load-misses" in line:
                    L1_dcache_load_misses = int(line.split(",")[1].strip())
                elif "L1-dcache-loads" in line:
                    L1_dcache_loads = int(line.split(",")[1].strip())
                elif "L1-dcache-store-misses" in line:
                    L1_dcache_store_misses = int(line.split(",")[1].strip())
                elif "L1-dcache-store" in line:
                    L1_dcache_stores = int(line.split(",")[1].strip())
                elif "l2_request_g1.all_no_prefetch" in line:
                    L2_all_demand_references = int(line.split(",")[1].strip())
                elif "l2_pf_hit_l2" in line:
                    L2_all_demand_references += int(line.split(",")[1].strip())
                elif "l2_pf_miss_l2_hit_l3" in line:
                    L2_all_demand_references += int(line.split(",")[1].strip())
                elif "l2_pf_miss_l2_l3" in line:
                    L2_all_demand_references += int(line.split(",")[1].strip())
                elif "l2_cache_req_stat.ic_dc_miss_in_l2" in line:
                    L2_all_demand_misses += int(line.split(",")[1].strip())
                elif "l2_pf_miss_l2_hit_l3" in line:
                    L2_all_demand_misses += int(line.split(",")[1].strip())
                elif "l2_pf_miss_l2_l3" in line:
                    L2_all_demand_misses += int(line.split(",")[1].strip())
            
        if L1_dcache_load_misses and L1_dcache_loads:
            cache_info["L1_Miss_Rate"] = L1_dcache_load_misses / L1_dcache_loads
        if L1_dcache_store_misses and L1_dcache_stores:
            cache_info["L1_Miss_Rate"] += L1_dcache_store_misses / L1_dcache_stores
        if L2_all_demand_misses and L2_all_demand_references:
            cache_info["L2_Miss_Rate"] = L2_all_demand_misses / L2_all_demand_references 

        return cache_info
    except Exception as e:
        return f"Error: {e}"

In [5]:
import pandas as pd
import datetime, time,threading, psutil

class SystemStatsGatherer:
    def __init__(self,  interval=2):
        self.temp_dict = {}
        self.not_firstime = False
        self.diskio_fieldname = ['read_count', 'write_count','read_bytes', 'write_bytes', 'read_time', 'write_time', 'read_merged_count', 'write_merged_count', 'busy_time']
        self.diskio_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0]
        self.swap_mem_fieldname = ['total', 'used', 'free', 'percent', 'sin', 'sout']
        self.swap_mem_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0]
        self.virtual_mem_fieldname = ['total', 'available','percent', 'used', 'free', 'active', 'inactive', 'buffers', 'cached', 'shared', 'slab']
        self.virtual_mem_fieldvals = [0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0 , 0.0, 0.0, 0.0, 0.0]
        self.tempsensor_fieldname = ['current','high', 'critical']
        self.tempsensor_fieldvals = [0.0 , 0.0, 0.0]
        self.samplerThread = threading.Thread(target=self.samplerThread)
        self.samplingInterval = interval
        self.stopThread = False


    def SampleTemp(self):

        tempsensor_name = ""
        res_temp = psutil.sensors_temperatures()
        res_diskio = psutil.disk_io_counters(perdisk=False, nowrap=True)
        res_swapmem = psutil.swap_memory()
        res_virtmem = psutil.virtual_memory()
        res_cpufreq = get_cpu_frequencies()
        res_psicpu = read_psi_info("cpu")
        res_psimem = read_psi_info("memory")
        res_psicio = read_psi_info("io")

        ctr = 0
        global not_firstime
        ts = datetime.datetime.now()
        if (self.not_firstime == False):
            self.temp_dict['time'] = [str(ts)]
        else:
            self.temp_dict['time'] += [str(ts)]

        ##get CPU frequencies
        if (self.not_firstime == False):
            for key, value in res_cpufreq.items():
                self.temp_dict[f'cpu_{key}'] = float(value)
        else:
            for key, value in res_cpufreq.items():
                self.temp_dict[f'cpu_{key}'] += float(value)

         ##get CPU PSI
        if (self.not_firstime == False):
            for key, value in res_psicpu.items():
                self.temp_dict[f'cpu_psi_{key}'] = value
        else:
            for key, value in res_psicpu.items():
                self.temp_dict[f'cpu_psi_{key}'] += value

         ##get Memory PSI
        if (self.not_firstime == False):
            for key, value in res_psimem.items():
                self.temp_dict[f'mem_psi_{key}'] = value
        else:
            for key, value in res_psimem.items():
                self.temp_dict[f'mem_psi_{key}'] += value

         ##get IO PSI
        if (self.not_firstime == False):
            for key, value in res_psicio.items():
                self.temp_dict[f'io_psi_{key}'] = value
        else:
            for key, value in res_psicio.items():
                self.temp_dict[f'io_psi_{key}'] += value
                
        ## get CPU Load averages
        loadavgs = [x / psutil.cpu_count() * 100 for x in psutil.getloadavg()]
        if (self.not_firstime == False):
            self.temp_dict['cpuloadavg_1min'] = loadavgs[0]
            self.temp_dict['cpuloadavg_5min'] = loadavgs[1]
            self.temp_dict['cpuloadavg_15min'] = loadavgs[2]
        else:
            self.temp_dict['cpuloadavg_1min'] += loadavgs[0]
            self.temp_dict['cpuloadavg_5min'] += loadavgs[1]
            self.temp_dict['cpuloadavg_15min'] += loadavgs[2]


        ## Process the Virtual Memory readings
        ctr = 0
        for val in res_virtmem:
            if (self.not_firstime == False):
                self.temp_dict['vm_'+self.virtual_mem_fieldname[ctr]] = [val]
            else:
                self.temp_dict['vm_'+self.virtual_mem_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the Swap Memory readings
        ctr = 0
        for val in res_swapmem:
            if (self.not_firstime == False):
                self.temp_dict['swap_'+self.swap_mem_fieldname[ctr]] = [val]
            else:
                self.temp_dict['swap_'+self.swap_mem_fieldname[ctr]] += [val]
            ctr += 1
        ## Process the DiskIO readings
        ctr = 0
        for val in res_diskio:
            if (self.not_firstime == False):
                self.temp_dict['diskio_'+self.diskio_fieldname[ctr]] = [val]
            else:
                self.temp_dict['diskio_'+self.diskio_fieldname[ctr]] += [val]
            ctr += 1
        # Process the Temperature readings
        for key,values in res_temp.items():
            # print (key,' => ',values)
            for temp_elems in values:
                # print (temp_elems)
                tempsensor_name = key
                ctr = 0;title_field = True
                for val in temp_elems:
                    if (title_field == True):
                        tempsensor_name +=  '-'+str(val)
                        title_field = False
                    else:
                        self.tempsensor_fieldvals[ctr] = val
                        ctr += 1
                        
                    # print(key,'-', '=>',tempsensor_fieldname[ctr],'-->',sensor_val)
                # print (tempsensor_name,'=>',tempsensor_fieldvals)
                ctr = 0
                for elems in self.tempsensor_fieldvals:
                    # print (tempsensor_name+'-'+tempsensor_fieldname[ctr],'=>',elems)
                    if (self.not_firstime == False):
                        self.temp_dict[tempsensor_name+'-'+self.tempsensor_fieldname[ctr]] = [elems]
                    else:
                        self.temp_dict[tempsensor_name+'-'+self.tempsensor_fieldname[ctr]] += [elems]
                    ctr += 1
        self.not_firstime = True

    def getReadings(self):
        return self.temp_dict
    
    def samplerThread(self):
        while (self.stopThread != True):
            self.SampleTemp()
            time.sleep(self.samplingInterval)
        return
    
    def startSampling(self):
        self.stopThread = False
        self.samplerThread.start()
        return 

    def stopSampling(self):
        self.stopThread = True
        self.samplerThread.join()
        return 


In [8]:
import torch.nn.utils.prune as prune
import numpy as np
import copy

import pandas as pd
import io
from torch.profiler import profile, record_function, ProfilerActivity


# COLUMN_NAMES = ['prune-config', 'prune-ratio','vm-percent','swap-percent','cpuloadavg-1min', 'cpuloadavg-5min', 'ratio','memory_consumption','acc']
df_final = pd.DataFrame()
load_model()
chipset_name = get_chipset_info()
arch = get_arch()
l1_size, l2_size = get_cache_sizes()
ddr_info = get_ddr_info()
gpu_stats = get_gpu_load_and_memory_used()
for ratio in np.arange(0.0,1.0,0.1):    
    statsMon = SystemStatsGatherer()
    statsMon.startSampling()
    ratio = round(ratio,2)
    print (ratio)
    parameters = ((model.conv1, "weight"),(model.conv2, "weight"),(model.fc1, "weight"),(model.fc2, "weight"),)
    prune.global_unstructured(parameters,pruning_method=prune.L1Unstructured,amount=ratio,)
    for items in parameters:
        # print(items[0],'=',items[1])
        prune.remove(items[0],items[1])

    prof = torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA, torch.profiler.ProfilerActivity],profile_memory=True, record_shapes=True,with_flops=True)
    print ('Profiling run started.')
    prof.start()
    acc = __test__(model)
    prof.stop()

    print ('Profiling run completed.\nCompiling result table')
    prof_result = prof.key_averages().table(sort_by='cpu_memory_usage', row_limit=100)
    prof_result_back = prof_result

    #Trim off the header and footer of the results
    prof_result_back = prof_result_back.rsplit("\n",3)[0]
    prof_result__ = ""
    for line in prof_result_back:
        if "-" not in line.split():
            prof_result__ += line 
    prof_result_back = prof_result__
    # print (prof_result_back)
    df = pd.read_csv(io.StringIO(prof_result_back), sep=r"\s\s+")

    # pd.set_option('display.max_rows', 500)
    # display(df)

    memory_consumption_str = df[(df['Name']=='[memory]')]['CPU Mem'].iloc[0]
    flops = df[(df['Name'] == 'aten::conv2d')]['Total FLOPs'].iloc[0]

    memory_consumption = 0
    if ('Gb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024*1024*1024
    elif ('Mb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024*1024
    elif ('Kb' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])*1024
    elif ('b' in memory_consumption_str):
        memory_consumption =  float(memory_consumption_str.split()[0])

    print(ratio, memory_consumption, flops, acc)
    statsMon.stopSampling()
    print('Collecting System Stats')
    df_stats = pd.DataFrame.from_dict(statsMon.getReadings())
    agg_df = df_stats[['vm_percent','swap_percent','cpuloadavg_1min', 'cpuloadavg_5min', 'cpu_0', 'cpu_1', 'cpu_psi_total', 'mem_psi_total', 'io_psi_total']]
    agg_df__ =  agg_df.mean()
    agg_df__['ratio'] = ratio
    agg_df__['memory_consumption'] = memory_consumption
    agg_df__['acc'] = acc
    agg_df__['prune_config'] = 'global_unstructured-L1Unstructured'
    agg_df__['prune_ratio'] = ratio
    agg_df__['flops'] = flops
    agg_df__['chipset_name'] = chipset_name
    agg_df__['arch'] = arch
    agg_df__['l1_size'] = l1_size
    agg_df__['l2_size'] = l2_size

    
    for i, ddr in enumerate(ddr_info):
        for key, value in ddr.items():
            column_name = f'{key}_{i+1}'
            agg_df__[column_name] = value

    for i, gpu_info in enumerate(gpu_stats):
        for key, value in gpu_info.items():
            column_name = f'{key}_{i+1}'
            agg_df__[column_name] = value
            

    display(agg_df__)
    df_final = pd.concat([df_final,agg_df__], axis=1)
    print('Aggregated stats')
    display(df_final)
    del prof_result
del model, loss_fn, optimizer



.
0.0
Profiling run started.


STAGE:2024-04-10 03:03:18 3675636:3675636 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-10 03:03:56 3675636:3675636 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-10 03:03:57 3675636:3675636 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


Profiling run completed.
Compiling result table


/tmp/ipykernel_3675636/970116782.py:47: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(io.StringIO(prof_result_back), sep=r"\s\s+")


0.0 188638822.4 48672000000.0 98.05499911308289


...

Aggregated stats


,0
vm_percent,67.103361
swap_percent,0.0
cpuloadavg_1min,4599.975
cpuloadavg_5min,4789.45
cpu_0,332198.007
cpu_1,331461.416
cpu_psi_total,25262427857751.0
mem_psi_total,163201829989.0
io_psi_total,212365812362.0
ratio,0.0


0.1
Profiling run started.


STAGE:2024-04-10 03:10:58 3675636:3675636 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-10 03:11:37 3675636:3675636 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-10 03:11:38 3675636:3675636 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


KeyboardInterrupt: 

In [ ]:
DF_SAVEFILENAME = '/Users/snigdhashekhar/PruningTest-dev/LeNet5-experiment.csv'
df_final_transpose = df_final.transpose().reset_index(drop=True)
display(df_final_transpose)
df_final_transpose.to_csv(DF_SAVEFILENAME, encoding='utf-8', sep=',')

,vm_percent,swap_percent,cpuloadavg_1min,cpuloadavg_5min,ratio,memory_consumption,acc,prune_config,prune_ratio,flops
0,84.859799,90.09799,7804.852295,7624.041748,0.0,3693671874.56,98.054999,global_unstructured-L1Unstructured,0.0,48672000000.0
1,84.629299,87.907643,5094.903564,5182.476807,0.1,3693671874.56,98.046666,global_unstructured-L1Unstructured,0.1,48672000000.0
2,84.62622,90.218293,5683.044434,5860.601807,0.2,3693671874.56,97.928333,global_unstructured-L1Unstructured,0.2,48672000000.0
3,85.228947,91.807237,6339.117432,5975.616455,0.3,3693671874.56,97.80333,global_unstructured-L1Unstructured,0.3,48672000000.0
4,85.115672,91.744776,4742.370605,4894.543457,0.4,3682934456.32,97.364998,global_unstructured-L1Unstructured,0.4,48672000000.0
5,85.251145,91.540458,5244.519043,5323.370361,0.5,3693671874.56,96.523333,global_unstructured-L1Unstructured,0.5,48672000000.0
6,85.458065,91.355484,5362.121582,5542.895508,0.6,3693671874.56,96.198332,global_unstructured-L1Unstructured,0.6,48672000000.0
7,85.247191,90.474157,6722.546387,6903.692627,0.7,3693671874.56,94.749999,global_unstructured-L1Unstructured,0.7,48672000000.0
8,85.222857,86.100714,5032.556152,5119.189453,0.8,3693671874.56,90.276664,global_unstructured-L1Unstructured,0.8,48672000000.0
9,85.302907,89.220349,6330.102539,6350.55542,0.9,3693671874.56,63.983333,global_unstructured-L1Unstructured,0.9,48672000000.0
